Our primary dataset:

Eurosat

[DOI](10.1109/IGARSS.2018.8519248)

[dataset](https://zenodo.org/records/7711810#.ZAm3k-zMKEA)


download EuroSAT_MS.zip (the full spectral dataset) and extract it into this directory

In [1]:
import torch
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
# from tqdm import tqdm
 
# !pip install torchvision
import torchvision
 
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
 
# !pip install torchmetrics
import torchmetrics
import os

x = torch.rand(5, 3)
print(x)

from sklearn.model_selection import train_test_split
import numpy as np
from pathlib import Path

tensor([[0.6863, 0.8538, 0.7062],
        [0.9087, 0.6379, 0.4148],
        [0.3764, 0.0230, 0.6513],
        [0.3884, 0.9568, 0.7225],
        [0.0031, 0.5742, 0.3412]])


In [ ]:
## Read in files and labels from datasets

# build the path for rgb image files
project_root = Path.cwd()
# pathlib automatically uses the correct separator for the OS.
image_dir_path = project_root / 'EuroSAT_RGB'
print(f"Loading data from: {image_dir_path}")

# Check if path is valid directory
if not image_dir_path.is_dir():
    print(f"Error: Directory not found at {image_dir_path}")
else:
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif']

# Gather all image file paths
all_image_paths = [file for file in image_dir_path.rglob('*') 
                 if file.suffix.lower() in image_extensions]
# gather labels for each image (label of image determined by directory housing image)
all_labels = [path.parent.name for path in all_image_paths]
print("First 5 rgb image file paths: ", all_image_paths[:5])
print("First 5 labels: ", all_labels[:5])

Loading data from: c:\Users\cokie\Workspace\GithubProjects\UrbanSatelliteAnalysis\EuroSAT_RGB
First 5 rgb image file paths:  [WindowsPath('c:/Users/cokie/Workspace/GithubProjects/UrbanSatelliteAnalysis/EuroSAT_RGB/EuroSAT_RGB/AnnualCrop/AnnualCrop_1.jpg'), WindowsPath('c:/Users/cokie/Workspace/GithubProjects/UrbanSatelliteAnalysis/EuroSAT_RGB/EuroSAT_RGB/AnnualCrop/AnnualCrop_10.jpg'), WindowsPath('c:/Users/cokie/Workspace/GithubProjects/UrbanSatelliteAnalysis/EuroSAT_RGB/EuroSAT_RGB/AnnualCrop/AnnualCrop_100.jpg'), WindowsPath('c:/Users/cokie/Workspace/GithubProjects/UrbanSatelliteAnalysis/EuroSAT_RGB/EuroSAT_RGB/AnnualCrop/AnnualCrop_1000.jpg'), WindowsPath('c:/Users/cokie/Workspace/GithubProjects/UrbanSatelliteAnalysis/EuroSAT_RGB/EuroSAT_RGB/AnnualCrop/AnnualCrop_1001.jpg')]
First 5 labels:  ['AnnualCrop', 'AnnualCrop', 'AnnualCrop', 'AnnualCrop', 'AnnualCrop']


In [3]:
## Get a list of unique data labels, and get a list of labels for all data in one-hot-encoding format

labels_unique = np.unique(all_labels)

label_tensor = []
label_index = []
for i in range(len(all_labels)):
   vect_init = np.zeros((1,len(labels_unique)))
   vect_init[0][np.where(labels_unique == all_labels[i])[0]] = 1
   label_tensor.append(vect_init)
   label_index.append(i)

print("all labels: ", labels_unique)
print("labels:", all_labels[0], all_labels[9000])
print("label index:", label_index[0],label_index[9000])
print("label tensor:", label_tensor[0],label_tensor[9000])

all labels:  ['AnnualCrop' 'Forest' 'HerbaceousVegetation' 'Highway' 'Industrial'
 'Pasture' 'PermanentCrop' 'Residential' 'River' 'SeaLake']
labels: AnnualCrop Highway
label index: 0 9000
label tensor: [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] [[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]


In [ ]:
# Split the file paths, using stratify to maintain class distribution

X_train, X_test, y_train, y_test = train_test_split(
    all_image_paths, label_index, test_size=0.2, random_state=42, stratify=all_labels
) 
print("First training input:", X_train[0])  
print("First training label (index of label in all_labels):", y_train[0])
print("Number of training inputs", len(X_train))
print("Number of training labels:", len(y_train))
print("Number of testing inputs", len(X_test))
print("Number of testing labels", len(y_test))

First training input: c:\Users\cokie\Workspace\GithubProjects\UrbanSatelliteAnalysis\EuroSAT_RGB\EuroSAT_RGB\AnnualCrop\AnnualCrop_1556.jpg
First training label (index of label in all_labels): 619
Number of training inputs 21600
Number of training labels: 21600
Number of testing inputs 5400
Number of testing labels 5400


In [13]:
from torch import nn
class CNN(nn.Module):
   def __init__(self, in_channels, num_classes):
 
       """
       Building blocks of convolutional neural network.
 
       Parameters:
           * in_channels: Number of channels in the input image (for grayscale images, 1)
           * num_classes: Number of classes to predict. In our problem, 10 (i.e digits from  0 to 9).
       """
       super(CNN, self).__init__()
 
       # 1st convolutional layer
       self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=3, padding=1)
       # Max pooling layer
       self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
       # 2nd convolutional layer
       self.conv2 = nn.Conv2d(in_channels=8, out_channels=1, kernel_size=3, padding=1)
       # Fully connected layer
       self.fc1 = nn.Linear(16 * 16, num_classes)

 
   def forward(self, x):
       """
       Define the forward pass of the neural network.
 
       Parameters:
           x: Input tensor.
 
       Returns:
           torch.Tensor
               The output tensor after passing through the network.
       """
       x = F.relu(self.conv1(x))  # Apply first convolution and ReLU activation
       x = self.pool(x)           # Apply max pooling
       x = F.relu(self.conv2(x))  # Apply second convolution and ReLU activation
       x = self.pool(x)           # Apply max pooling
       x = x.reshape(x.shape[0], -1)  # Flatten the tensor
       x = self.fc1(x)            # Apply fully connected layer
       x = torch.nn.functional.softmax(x,dim=1) # apply softmax to x
       return x



In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CNN(in_channels=3, num_classes=10).to(device)
print(model)
# >>> CNN(
#   (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
#   (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
#   (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
#   (fc1): Linear(in_features=784, out_features=10, bias=True)
# )

CNN(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=256, out_features=10, bias=True)
)


In [ ]:
# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [108]:
# import tqdm
from PIL import Image
from torchvision.transforms import ToTensor

torch.backends.nnpack.enabled = False

num_epochs=10
for epoch in range(num_epochs):
  # Iterate over training batches
  print(f"Epoch [{epoch + 1}/{num_epochs}]")

  for image,li in zip(X_train,y_train):
    image = Image.open(image)
    image = ToTensor()(image)
    label = ToTensor()(label_tensor[li])
    # print(image)
    # print(label)
    # data = image.to(device)
    # targets = label.to(device)
    scores = model(image)
    # print("scores:",scores)
    # print("label:",label)
    loss = criterion(scores, label[0])
    optimizer.zero_grad()
    loss.backward()
  optimizer.step()

Epoch [1/10]


[W930 17:30:13.649572041 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W930 17:30:13.659122743 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W930 17:30:13.664841223 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W930 17:30:13.667738749 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W930 17:30:13.675225954 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W930 17:30:13.686449850 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W930 17:30:13.690439640 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W930 17:30:13.694837724 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W930 17:30:13.700723640 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W930 17:30:13.709518550 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W930 17:3

KeyboardInterrupt: 